In [1]:
import ee
import geemap
import sys,os
sys.path.append(r'E:\09_Code\Gis_Script\GEE_Func')
from S1_distor_dedicated import load_S1collection,S1_CalDistor,DEM_caculator
from S2_filter import merge_s2_collection
from GEE_DataIOTrans import BandTrans,DataTrans,DataIO
from GEE_CorreterAndFilters import S1Corrector
import math
from functools import partial
from tqdm import tqdm 

# 重载函数
# import importlib
# importlib.reload(S1_distor_dedicated)

geemap.set_proxy(port=10809)
# ee.Authenticate()
ee.Initialize()

d:\Code_base\anaconda\envs\GEE\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
d:\Code_base\anaconda\envs\GEE\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


torch or torchvision do not import
GEE_DataIOTrans not support gdal


In [2]:
# 获取S1 、 DEM
AOI =  ee.Geometry.Polygon([[[93.15172089782506, 30.60689894788499],
   [93.20966268713462, 30.60689894788499],
   [93.20966268713462, 30.63625137380944],
   [93.15172089782506, 30.63625137380944],
   [93.15172089782506, 30.60689894788499]]])

year = '2019'
START_DATE  = ee.Date(year + '-01-01')
END_DATE   = ee.Date(year + '-12-30')
TIME_LEN   = END_DATE.difference(START_DATE, 'days').abs()
MIDDLE_DATE = START_DATE.advance(TIME_LEN.divide(ee.Number(2)).int(),'days')
Origin_scale = 10
Prj_scale = 30


s1_ascending, s1_descending = load_S1collection(AOI,START_DATE,END_DATE,MIDDLE_DATE,FilterSize=30)
DEMSRTM = ee.Image('USGS/SRTMGL1_003')
DEM_prj = DEMSRTM.projection()

S1_image = s1_ascending
DEM = DEMSRTM
Projection = S1_image.select(0).projection()
Mask = S1_image.select(0).mask()

# 获取辅助线
azimuthEdge, rotationFromNorth, startpoint, endpoint, coordinates_dict  = S1Corrector.getS1Corners(s1_ascending, AOI,'ASCENDING')

Heading = azimuthEdge.get('azimuth')
s1_azimuth_across = ee.Number(Heading).subtract(90.0) # 距离向
Auxiliarylines = ee.Geometry.LineString([startpoint, endpoint])

Eq_pixels = DataTrans.Eq_pixels

BandTrans.delBands(S1_image, ['VH', 'VV'])
Cal_image = (Eq_pixels(BandTrans.delBands(S1_image, 'VV','VH').resample('bicubic')).rename('angle')
                      .addBands(ee.Image.pixelCoordinates(Projection))
                      .addBands(DEM.select('elevation'))
                      .addBands(ee.Image.pixelLonLat())
                      .updateMask(Mask)
                      .reproject(crs=Projection, scale=Prj_scale)
                      .clip(AOI))
Templist_A = S1_CalDistor.AuxiliaryLine2Point(Cal_image, s1_azimuth_across, coordinates_dict, Auxiliarylines, Origin_scale)

Without Filter


In [80]:
import numpy as np
from scipy.optimize import least_squares
from tqdm import tqdm

# 面转线-->获取4、9邻域-->计算代求点高程

def Line2Points(feature,region,scale=30):
    # 从Feature中提取线几何对象
    line_geometry = ee.Feature(feature).geometry().intersection(region)
    
    # 获取线的所有坐标点
    coordinates = line_geometry.coordinates()
    start_point = ee.List(coordinates.get(0)) # 起点坐标
    end_point = ee.List(coordinates.get(-1))  # 终点坐标
    
    # 获取线段的总长度
    length = line_geometry.length()

    # 计算插入点的数量
    num_points = length.divide(scale).subtract(1).floor()

    # 计算每个间隔点的坐标
    def interpolate(i):
        i = ee.Number(i)
        fraction = i.divide(num_points)
        interpolated_lon = ee.Number(start_point.get(0)).add(
            ee.Number(end_point.get(0)).subtract(ee.Number(start_point.get(0))).multiply(fraction))
        interpolated_lat = ee.Number(start_point.get(1)).add(
            ee.Number(end_point.get(1)).subtract(ee.Number(start_point.get(1))).multiply(fraction))
        return ee.Feature(ee.Geometry.Point([interpolated_lon, interpolated_lat]))

    # 使用条件表达式过滤长度为0的线段
    filtered_points = ee.FeatureCollection(ee.Algorithms.If(num_points.gt(0), 
                                                            ee.FeatureCollection(ee.List.sequence(1, num_points).map(interpolate)),
                                                            ee.FeatureCollection([])))

    return filtered_points

def map_function(item,len=3):
    '''该函数len=3起步判断，由于len=2的时候会将单点判定为正确，故存在误差
     为了节省计算量，未对此处进行修改'''
    item_list = ee.List(item)
    return ee.Algorithms.If(item_list.size().gte(len), item_list, None)

def get_neighborhood_info(point,image,scale=30,neighborhood_type='4'):
    
    point = ee.Geometry.Point(point)
    buffer_region = point.buffer(30)

    # 生成矩形区域
    region = buffer_region.bounds()
    # 获取矩形的坐标
    coords  = ee.List(region.coordinates().get(0))

    # 提取四个角点坐标
    corner1 = ee.Geometry.Point(ee.List(coords.get(0)))
    corner2 = ee.Geometry.Point(ee.List(coords.get(1)))
    corner3 = ee.Geometry.Point(ee.List(coords.get(2)))
    corner4 = ee.Geometry.Point(ee.List(coords.get(3)))
    
    if neighborhood_type=='4':
        # 创建包含所有点的MultiPoint几何对象
        region_coord = ee.Geometry.MultiPoint([
                                                corner1.coordinates(), corner2.coordinates(), 
                                                corner3.coordinates(), corner4.coordinates()])
    
    elif neighborhood_type=='9': 
        # 创建线段并计算边的中心点
        edge_center1 = ee.Geometry.LineString([corner1.coordinates(), corner2.coordinates()]).centroid()
        edge_center2 = ee.Geometry.LineString([corner2.coordinates(), corner3.coordinates()]).centroid()
        edge_center3 = ee.Geometry.LineString([corner3.coordinates(), corner4.coordinates()]).centroid()
        edge_center4 = ee.Geometry.LineString([corner4.coordinates(), corner1.coordinates()]).centroid()

        # 创建包含所有点的MultiPoint几何对象
        region_coord = ee.Geometry.MultiPoint([
                        corner1.coordinates(), corner2.coordinates(), 
                        corner3.coordinates(), corner4.coordinates(),
                        edge_center1.coordinates(), edge_center2.coordinates(), 
                        edge_center3.coordinates(), edge_center4.coordinates(),
                        point.coordinates()])
    else:
        print('neighborhood_type should be "4" or "9"')

    # 使用MultiPoint几何对象作为geometry参数
    result = image.reduceRegion(
        reducer=ee.Reducer.toList(),  # 转换为列表
        geometry=region_coord,
        scale=scale,
        maxPixels=1e9)

    # 在结果中添加点坐标
    result = result.set('point_coordinates', point.coordinates())
    
    return result 

def weighted_avg_func(neighbor):
    # 使用ee.Number转换index，因为原始的index是ee.ComputedObject
    neighbors_info = ee.Dictionary(neighbor)

    # 将邻域点信息转换为ee.Array
    x_coords = ee.Array(neighbors_info.get('longitude'))
    y_coords = ee.Array(neighbors_info.get('latitude'))
    elevations = ee.Array(neighbors_info.get('elevation'))
    angles = ee.Array(neighbors_info.get('angle'))
    point_coords  = ee.List(neighbors_info.get('point_coordinates'))
    # 计算距离和权重
    point1_x = ee.Number(point_coords.get(0))
    point1_y = ee.Number(point_coords.get(1))
    distances = x_coords.subtract(point1_x).pow(2).add(y_coords.subtract(point1_y).pow(2)).sqrt()
    weights = distances.pow(-1)
    sum_weights = weights.reduce(ee.Reducer.sum(), [0])

    # 计算加权平均高程
    weighted_elevations = elevations.multiply(weights)
    weighted_avg_elevation = weighted_elevations.reduce(ee.Reducer.sum(), [0]).divide(sum_weights).get([0])

    # 计算平均角度
    weighted_angles = elevations.multiply(angles)
    weighted_avg_angles = weighted_angles.reduce(ee.Reducer.sum(), [0]).divide(sum_weights).get([0])

    return ee.Dictionary({'elevation': weighted_avg_elevation,'angle':weighted_avg_angles})

def avg_func(neighbor):
    neighbors_info = ee.Dictionary(neighbor)
    elevations = ee.Array(neighbors_info.get('elevation'))
    angles = ee.Array(neighbors_info.get('angle'))
    return ee.Dictionary({'elevation':elevations.reduce(ee.Reducer.mean(),[0]).get([0]) ,
                           'angle':angles.reduce(ee.Reducer.mean(),[0]).get([0])})

def Volum9_func(neighbors_info):
    
    # 修改方程函数以匹配新的模型
    def equation(params, x, y):
        a, b, c, d, e, f = params
        return a*x**2 + b*y**2 + c*x*y + d*x + e*y + f

    # 修改残差函数，去除z作为变量
    def residuals(params, x, y, z_true):
        z_pred = equation(params, x, y)
        return z_pred - z_true
    
    # 根据求得的参数和给定点的经纬度计算高程
    def calculate_z(params, x, y):
        # 根据新模型参数更新，只有6个参数
        a, b, c, d, e, f = params
        # 更新方程以反映新的模型形式
        z_pred = a*x**2 + b*y**2 + c*x*y + d*x + e*y + f
        return z_pred
    

    lon = np.array(neighbors_info['longitude'])
    lat = np.array(neighbors_info['latitude'])
    elevation = np.array(neighbors_info['elevation'])
    angles = np.array(neighbors_info['angle'])
    lon_pre, lat_pre = np.array(neighbors_info['point_coordinates'])

    # 初始参数猜测
    initial_guess = np.zeros(6)

    # 使用最小二乘法求得的参数,预测高程
    result0 = least_squares(residuals, initial_guess, args=(lon, lat, elevation))
    params0 = result0.x  
    z_pred = calculate_z(params0, lon_pre, lat_pre)

    # 使用最小二乘法求得的参数,预测angle
    result1 = least_squares(residuals, initial_guess, args=(lon, lat, angles))
    params1 = result1.x  
    angle_pred = calculate_z(params1, lon_pre, lat_pre)
    return {'elevation':z_pred,'angle':angle_pred}

def Flat4_func(neighbors_info):

    lon = neighbors_info['longitude']
    lat = neighbors_info['latitude']
    elevation = neighbors_info['elevation']
    angles = np.array(neighbors_info['angle'])
    x_pred, y_pred = np.array(neighbors_info['point_coordinates'])
    
    A = np.vstack([lon, lat, np.ones(len(lon))]).T  # 构建矩阵A

    b = elevation  # 高程向量b
    # 使用最小二乘法解方程Ax = b
    x, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
    # x包含拟合参数a, b, c（注意这里不再是1/c）
    a, b, c = x
    z_pred = a * x_pred + b * y_pred + c

    b = angles
    x, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
    a, b, c = x
    angle_pred = a * x_pred + b * y_pred + c

    return {'elevation':z_pred,'angle':angle_pred}

def apply_map_to_list(input_list, func): return input_list.map(func)

def apply_map_to_list_local(input_list, func): return map(func,input_list)

Len_Templist_A =  Templist_A.size().getInfo()
pbar = tqdm(range(Len_Templist_A),ncols=80)
Neighbors = '4' # '9'

# 需要注意'Area_elavation','Volum_elavation'涉及数学拟合，GEE-->Python环境
Elvevation_model = 'weighted_avg_elevation' #'weighted_avg_elevation' , 'avg_elevation', 'Area_elavation','Volum_elavation'

All_PointLine = [] 
for i in pbar:
    points = Line2Points(Templist_A.get(i),region=AOI,scale=Prj_scale)  
    points = points.geometry().coordinates()
    All_PointLine.append(points)
    
EE_PointLine = ee.List(All_PointLine[110:111])

# 使用 map() 方法应用映射函数
EE_PointLine = EE_PointLine.map(partial(map_function,len=3)).removeAll([None])

cal_neighbors = apply_map_to_list(EE_PointLine, 
                               lambda x: ee.List(x).map(partial(get_neighborhood_info,image=Cal_image,
                                                                scale=Prj_scale,neighborhood_type=Neighbors)))

if Elvevation_model == 'weighted_avg_elevation':
    points_elevations_neighborhood = apply_map_to_list(
                                        cal_neighbors,      
                                        lambda x: ee.List(x).map(weighted_avg_func))
    
elif Elvevation_model == 'avg_elevation':
    points_elevations_avg = apply_map_to_list(
                                        cal_neighbors,
                                        lambda x: ee.List(x).map(avg_func))
    
elif (Elvevation_model == 'Area_elavation') or (Elvevation_model == 'Volum_elavation'):

    neighbors_info = cal_neighbors.getInfo() # 非常大
    if Elvevation_model == 'Area_elavation':
        points_elevations_area = list(apply_map_to_list_local(neighbors_info,lambda x: list(map(Flat4_func,x))))
    elif Elvevation_model == 'Volum_elavation':
        points_elevations_volum = list(apply_map_to_list_local(neighbors_info,lambda x: list(map(Volum9_func,x))))

100%|█████████████████████████████████████████| 557/557 [00:07<00:00, 75.97it/s]


In [95]:
neighbor = ee.List(cal_neighbors.get(0)).get(0)

In [ ]:
# 使用ee.Number转换index，因为原始的index是ee.ComputedObject
neighbors_info = ee.Dictionary(neighbor)

# 将邻域点信息转换为ee.Array
x_coords = ee.Array(neighbors_info.get('longitude'))
y_coords = ee.Array(neighbors_info.get('latitude'))
elevations = ee.Array(neighbors_info.get('elevation'))
angles = ee.Array(neighbors_info.get('angle'))
point_coords  = ee.List(neighbors_info.get('point_coordinates'))
# 计算距离和权重
point1_x = ee.Number(point_coords.get(0))
point1_y = ee.Number(point_coords.get(1))
distances = x_coords.subtract(point1_x).pow(2).add(y_coords.subtract(point1_y).pow(2)).sqrt()
weights = distances.pow(-1)
sum_weights = weights.reduce(ee.Reducer.sum(), [0])

# 计算加权平均高程
weighted_elevations = elevations.multiply(weights)
weighted_avg_elevation = weighted_elevations.reduce(ee.Reducer.sum(), [0]).divide(sum_weights)

# 计算平均角度
weighted_angles = elevations.multiply(angles)
weighted_avg_angles = weighted_angles.reduce(ee.Reducer.sum(), [0]).divide(sum_weights)

In [128]:
weighted_avg_elevation.get([0])

In [127]:
ee.Array([8]).get([0])

In [49]:
import json
# 将数据转换为 JSON 格式并保存到文件中
with open('neighbors_info.json', 'w') as json_file:
    json.dump(neighbors_info, json_file, indent=4)

In [60]:
list_of_dicts = ee.List([
    {'angle': 10, 'elevation': 100},
    {'angle': 20, 'elevation': 150},
    {'angle': 30, 'elevation': 200},
    {'angle': 40, 'elevation': 250},
    {'angle': 50, 'elevation': 300},
    # 添加更多字典...
])

In [83]:
list_of_dicts

In [ ]:
# 滤波点。 根据高程和距离？ ，检测高差？